## Gracefull Shutdown
***

Vamos pegar a situação que você está rodando um script que consome uma fila e realiza um processo longo, temos 10 PODs rodando atualmente nesse sistema, a medida que os PODS vão aumentando e diminuindo precisamos de um mecanismo de reciclagem de PODs inteligente o suficiente para não matar um processo no meio do caminho, esperar ele terminar e depois reciclar seu POD.

Para podemos implementar um gracefull shutdown com signals.

In [1]:
import asyncio
import signal
import random

In [2]:
stop_event = asyncio.Event()

In [3]:
def shutdown(sig: signal.Signals) -> None:
    """
    Setando o evento de parada.
    """

    print(f"Sinal de sair recebido: {sig.name}")
    stop_event.set()
    print("Evento de parada setado.")

In [4]:
def setup_signal_handler() -> None:
    """
    Observador de sinais como o Ctrl + C para parada
    do script ou graceful shutdown.
    """

    loop = asyncio.get_running_loop()

    # kill -l (mostra todos os comandos de sinais)
    # SIGKILL -> Mata o processo forçado e não tem oq fazer (kill -9)
    # SIGHUP -> Recebeu um evento de desligamento do processo (desescalonamento de um pod)
    # SIGTERM -> Recebeu um evento para terminar o processo (parada de um pod)
    # SIGINT -> Recebeu um evento de interrupção (Ctrl+C)
    # Se ocorrer qualquer um desses sinais execute o evento de parada (shutdown)
    for sig in (signal.SIGHUP, signal.SIGTERM, signal.SIGINT):
        loop.add_signal_handler(sig, shutdown, sig)

In [5]:
async def worker(n) -> None:
    """
    Trabalhador processando algo de forma simulada com o
    sleep de 1 a 15 de forma randomica.
    """

    while True:
        print(f"Worker {n}")
        if stop_event.is_set():
            print(f"Worker {n}: Evento de parada setado. Parando.")
            break

        await asyncio.sleep(random.randint(1, 15))

In [6]:
async def main() -> None:
    """
    Tetando o graceful shutdown.
    """

    setup_signal_handler()

    # Criando 6 tarefas
    tasks = []
    for n in range(6):
        task = worker(n)
        tasks.append(task)

    await asyncio.gather(*tasks)

In [7]:
await main()

Worker 0
Worker 1
Worker 2
Worker 3
Worker 4
Worker 5
Worker 1
Worker 1
Worker 2
Worker 5
Worker 4
Worker 3
Worker 0
Worker 5
Worker 2
Worker 1
Worker 5
Worker 4
Worker 5
Sinal de sair recebido: SIGINT
Evento de parada setado.
Worker 0
Worker 0: Evento de parada setado. Parando.
Worker 3
Worker 3: Evento de parada setado. Parando.
Worker 4
Worker 4: Evento de parada setado. Parando.
Worker 2
Worker 2: Evento de parada setado. Parando.
Worker 1
Worker 1: Evento de parada setado. Parando.
Worker 5
Worker 5: Evento de parada setado. Parando.
